## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Two Rivers,US,44.15,-87.57,80.01,61,20,8.05,few clouds
1,1,Hobart,AU,-42.88,147.33,45.00,73,75,10.29,broken clouds
2,2,Constitucion,CL,-35.33,-72.42,51.40,71,0,4.94,clear sky
3,3,Meulaboh,ID,4.14,96.13,78.33,85,100,3.80,moderate rain
4,4,Puerto Ayora,EC,-0.74,-90.35,66.99,87,68,10.00,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [5]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & \
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Two Rivers,US,44.15,-87.57,80.01,61,20,8.05,few clouds
3,3,Meulaboh,ID,4.14,96.13,78.33,85,100,3.80,moderate rain
6,6,Kavieng,PG,-2.57,150.80,84.06,73,20,9.19,few clouds
9,9,Albany,US,42.60,-73.97,75.99,49,9,1.99,clear sky
11,11,Sao Geraldo Do Araguaia,BR,-6.40,-48.55,83.93,25,0,2.68,clear sky
13,13,Umm Lajj,SA,25.02,37.27,87.73,56,0,9.08,clear sky
18,18,Kodiak,US,57.79,-152.41,75.20,38,1,12.75,clear sky
19,19,Avera,US,33.19,-82.53,80.60,51,40,4.70,light rain
22,22,Kimbe,PG,-5.55,150.14,83.44,65,67,3.56,broken clouds
27,27,Port Blair,IN,11.67,92.75,81.90,80,100,10.11,light rain


In [6]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()
preferred_cities_df.notnull().count()

City_ID                267
City                   267
Country                267
Lat                    267
Lng                    267
Max Temp               267
Humidity               267
Cloudiness             267
Wind Speed             267
Current Description    267
dtype: int64

In [7]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
print("No null rows")

No null rows


In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Two Rivers,US,80.01,few clouds,44.15,-87.57,
3,Meulaboh,ID,78.33,moderate rain,4.14,96.13,
6,Kavieng,PG,84.06,few clouds,-2.57,150.80,
9,Albany,US,75.99,clear sky,42.60,-73.97,
11,Sao Geraldo Do Araguaia,BR,83.93,clear sky,-6.40,-48.55,
13,Umm Lajj,SA,87.73,clear sky,25.02,37.27,
18,Kodiak,US,75.20,clear sky,57.79,-152.41,
19,Avera,US,80.60,light rain,33.19,-82.53,
22,Kimbe,PG,83.44,broken clouds,-5.55,150.14,
27,Port Blair,IN,81.90,light rain,11.67,92.75,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    params["location"] = f"{lat},{lng}"
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    
    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except:
        print("Hotel not found... skipping.")
        

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df["Hotel Name"].isnull()
hotel_df.head(10)
clean_hotel_df = hotel_df.loc[hotel_df["Hotel Name"] != ""]
clean_hotel_df.isnull().count()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Two Rivers,US,80.01,few clouds,44.15,-87.57,Lighthouse Inn On Lake Michigan
3,Meulaboh,ID,78.33,moderate rain,4.14,96.13,Kost Pria
6,Kavieng,PG,84.06,few clouds,-2.57,150.80,Nusa Island Retreat
11,Sao Geraldo Do Araguaia,BR,83.93,clear sky,-6.40,-48.55,Hotel Rio Araguaia
13,Umm Lajj,SA,87.73,clear sky,25.02,37.27,Moon light Furnished Units
18,Kodiak,US,75.20,clear sky,57.79,-152.41,Best Western Kodiak Inn and Convention Center
22,Kimbe,PG,83.44,broken clouds,-5.55,150.14,Kimbe Bay Hotel
27,Port Blair,IN,81.90,light rain,11.67,92.75,Fortune Resort Bay Island
28,Dong Hoi,VN,78.21,overcast clouds,17.48,106.60,Nam Long Hotel
31,Isangel,VU,82.40,few clouds,-19.55,169.27,Volcano Sea View Tree House


In [11]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [12]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>

"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.75)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))